In [ ]:
from numpy.polynomial import Polynomial
from phe import paillier

In [ ]:
def set_intersection(players):
    for i in players:
        f[i] = (x - S[i][1])...(x - S[i][k])
        for player in range(i + 1, i + c + 1):
            player.send(f[i])
        